In [1]:
!pip install opensmile gdown transformers soundfile --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 996.0/996.0 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.4/138.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.8/324.8 kB 14.2 MB/s eta 0:00:00


### Accessing Dataset

In [6]:
import os
import tarfile
from pathlib import Path
import os
import tempfile
from pathlib import Path
import matplotlib.pyplot as plt
import librosa
import librosa.display
from IPython.display import Audio, display
import pandas as pd
import numpy as np
import soundfile as sf
from transformers import pipeline
import torch

# install dataset using gdown
!gdown 1p4ZQOwbHkD2RAvq2K5ekY5gcMi24XLnS

tarball_filename = "ADReSSo21-diagnosis-train.tar"

# extract contents from tarball
with tarfile.open(tarball_filename, 'r:*') as tar:
    tar.extractall(path="./")

# remove tarball after extraction
os.remove(tarball_filename)

Downloading...
From (original): https://drive.google.com/uc?id=1p4ZQOwbHkD2RAvq2K5ekY5gcMi24XLnS
From (redirected): https://drive.google.com/uc?id=1p4ZQOwbHkD2RAvq2K5ekY5gcMi24XLnS&confirm=t&uuid=886a0c64-f086-4a34-8023-f47f964b6886
To: /content/ADReSSo21-diagnosis-train.tar
100% 1.75G/1.75G [00:33<00:00, 52.8MB/s]


In [11]:
def get_dataset_paths(dataset_root: str = "ADReSSo21/diagnosis/train"):
    """
    return a dictionary with relevant dataset paths.
    """
    dataset_dir = Path(dataset_root)
    paths = {
        'audio': dataset_dir / 'audio',
        'audio_ad': dataset_dir / 'audio' / 'ad',
        'audio_cn': dataset_dir / 'audio' / 'cn',
        'segmentation': dataset_dir / 'segmentation',
        'segmentation_ad': dataset_dir / 'segmentation' / 'ad',
        'segmentation_cn': dataset_dir / 'segmentation' / 'cn'
    }
    return paths

def load_audio_file(file_path: Path):
    """load and return the audio waveform and sampling rate using soundfile."""
    return sf.read(file_path)

def load_segmentation(seg_file: Path):
    """load and return the segmentation csv as a pandas dataframe."""
    return pd.read_csv(seg_file)

def extract_patient_segments(audio: np.ndarray, sr: int, seg_df: pd.DataFrame, speaker: str = "PAR"):
    """
    extract patient segments (where speaker == speaker) from the audio.

    returns:
      - patient_mask: array with patient segments (nan elsewhere).
      - concatenated: patient segments concatenated into one array.
      - segments: list of (begin, end) sample index tuples.
    """
    patient_df = seg_df[seg_df['speaker'] == speaker]
    patient_mask = np.full_like(audio, np.nan)
    segments_list = []
    segments = []

    for _, row in patient_df.iterrows():
        begin_sample = int(float(row['begin']) * sr / 1000)
        end_sample = int(float(row['end']) * sr / 1000)
        patient_mask[begin_sample:end_sample] = audio[begin_sample:end_sample]
        segments_list.append(audio[begin_sample:end_sample])
        segments.append((begin_sample, end_sample))

    concatenated = np.concatenate(segments_list) if segments_list else np.array([])
    return patient_mask, concatenated, segments

def process_audio(audio_file: Path, seg_file: Path, plot: bool = False):
    """
    load an audio file and its corresponding segmentation csv,
    extract patient segments (concatenated), and optionally plot the overlay.

    returns:
      audio, sr, patient_mask, concatenated (patient-only audio), segments
    """
    audio, sr = load_audio_file(audio_file)
    seg_df = load_segmentation(seg_file)
    patient_mask, concatenated, segments = extract_patient_segments(audio, sr, seg_df)

    if plot:
        time_axis = np.linspace(0, len(audio) / sr, num=len(audio))
        plt.figure(figsize=(14, 4))
        plt.plot(time_axis, audio, label="original")
        plt.plot(time_axis, patient_mask, label="patient-only")
        plt.xlabel("time (s)")
        plt.ylabel("amplitude")
        plt.title("patient speech isolation")
        plt.legend()
        plt.show()

    return audio, sr, patient_mask, concatenated, segments

def init_transcriber(model_name: str = "openai/whisper-large", device: int = -1):
    """
    initialize and return the asr pipeline from hugging face's transformers.

    use device=-1 for cpu or set device=0 (or appropriate index) for gpu.
    """
    transcriber = pipeline("automatic-speech-recognition", model=model_name, device=device)
    return transcriber

def transcribe_audio_file(file_path: str, transcriber) -> str:
    """
    transcribe an audio file given its file path using the provided transcriber.

    returns:
      the transcription text.
    """
    transcription_result = transcriber(file_path, return_timestamps=True)
    return transcription_result.get("text", "")

def create_transcription_df(transcription_records: list) -> pd.DataFrame:
    """
    create a pandas dataframe from a list of transcription records.
    """
    return pd.DataFrame(transcription_records)

def run_single_audio_transcription():
    paths = get_dataset_paths()

    # set device: use 0 for gpu (or -1 for cpu)
    device = 0
    print(f"device set to use {'cpu' if device == -1 else 'gpu'}.")
    transcriber = init_transcriber(model_name="openai/whisper-large", device=device)

    # pick a single audio file from the audio_ad group
    audio_file = next(paths['audio_ad'].glob("*.wav"), None)
    if audio_file is None:
        raise FileNotFoundError("no audio file found in " + str(paths['audio_ad']))

    # construct the corresponding segmentation csv file path
    seg_file = paths['segmentation_ad'] / f"{audio_file.stem}.csv"

    # process audio to extract patient-only segments
    audio, sr, patient_mask, concatenated, segments = process_audio(audio_file, seg_file, plot=False)

    if concatenated.size == 0:
        print("no patient speech segments found in the file.")
        return
    else:
        print("transcribing patient-only speech for file:", audio_file.name)

        # write concatenated patient audio to a temporary wav file
        import tempfile
        with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as tmp_file:
            temp_filename = tmp_file.name
        sf.write(temp_filename, concatenated, sr)

        # transcribe using the temporary file
        transcription_text = transcribe_audio_file(temp_filename, transcriber)
        print("patient-only transcription:")
        print(transcription_text)

        # remove the temporary file
        os.remove(temp_filename)

        # save the transcription in a csv file
        record = [{"file_name": audio_file.name, "transcription": transcription_text}]
        df = create_transcription_df(record)
        csv_filename = "patient_transcriptions.csv"
        df.to_csv(csv_filename, index=False)
        print(f"patient-only transcription saved to {csv_filename}")


### Transcription


In [10]:
if __name__ == "__main__":
    run_single_audio_transcription()

Device set to use gpu.


Device set to use cuda:0


Transcribing patient-only speech for file: adrso049.wav


/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


Patient-only Transcription:
 The little boy climbing up in some cookers out the cooking job and his little sister reach of a song and they watch Dianna on the loud and it makes us to watch in the basement at the sink Big sis washing the dishes and she got dishes sitting on the sink. I think she's running water. And I say the little sister's reach. Johnny Johnny he's he's up on the ladder, get getting some cookies in the. This is the region that we should reach it up after. He's passing down to her. and the lather about to turn over. The cups, maybe she done washed them and she got them sitting on the sink. Maybe running water on the sink and she got a curry to pour. That she might get some light in there. Since the dishes stacked up, they might be on this thing. Nothing but alright.
Patient-only transcription saved to patient_transcriptions.csv
